<a href="https://colab.research.google.com/github/nourhan254/Autism-/blob/main/CNN_Alzheimer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Imports
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import cv2
import os

# Paths
train_path = '/content/drive/MyDrive/ProcessedImage/training_set'
test_path = '/content/drive/MyDrive/ProcessedImage/test_set'

# Image size and batch
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1)
])

# Load training data with augmentation
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=True
).map(lambda x, y: (data_augmentation(x) / 255.0, y)).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Load validation (test) data
val_ds = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='binary',
    shuffle=False
).map(lambda x, y: (x / 255.0, y)).cache().prefetch(buffer_size=tf.data.AUTOTUNE)

# Build the model using MobileNetV2
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False

inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

# Compile
model.compile(optimizer=Adam(0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Class weights to handle imbalance (Normal: 315, AD: 207)
class_weight = {0: 1.52, 1: 1.0}

# Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=1e-6)

# Train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    class_weight=class_weight,
    callbacks=[early_stop, reduce_lr]
)


Mounted at /content/drive
Found 522 files belonging to 2 classes.
Found 51 files belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 78s 4s/step - accuracy: 0.5620 - loss: 0.9176 - val_accuracy: 0.6863 - val_loss: 0.6698 - learning_rate: 1.0000e-04
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 29s 2s/step - accuracy: 0.5019 - loss: 0.9273 - val_accuracy: 0.5882 - val_loss: 0.6736 - learning_rate: 1.0000e-04
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5034 - loss: 0.8874 - val_accuracy: 0.5490 - val_loss: 0.6739 - learning_rate: 1.0000e-04
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 43s 2s/step - accuracy: 0.5158 - loss: 0.8931 - val_accuracy: 0.5490 - val_loss: 0.6733 - learning_rate: 2.0000e-05
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.5265 - loss: 0.8755 - val_accuracy: 0.5490 - val_loss: 0.6730 - learning_rate: 2.0000e-05


In [4]:
# Path to a test image
img_path = '/content/drive/MyDrive/ProcessedImage/test.png'  # change as needed

# Load and preprocess
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img_resized = tf.image.resize(img, IMG_SIZE)
img_normalized = img_resized / 255.0
img_expanded = np.expand_dims(img_normalized, axis=0)

# Predict
pred = model.predict(img_expanded)
label = 'AD' if pred[0][0] > 0.5 else 'Normal'
print(f"Predicted Class: {label} ({pred[0][0]:.4f})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Predicted Class: AD (0.7186)
